In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import os
import json

# 自动下载模型时，指定使用modelscope; 否则，会从HuggingFace下载
os.environ['VLLM_USE_MODELSCOPE'] = 'True'

def get_completion(prompts, model, tokenizer=None, temperature=0.6, top_p=0.95, top_k=20, min_p=0, max_tokens=4096, max_model_len=8192):
    stop_token_ids = [151645, 151643]
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        min_p=min_p,
        max_tokens=max_tokens,
        stop_token_ids=stop_token_ids
    )
    llm = LLM(model=model, tokenizer=tokenizer, max_model_len=max_model_len,trust_remote_code=True)  # max_model_len 用于限制模型在推理过程中可以处理的最大输入和输出长度之和。
    outputs = llm.generate(prompts, sampling_params=sampling_params)
    return outputs

/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-03 06:06:46 [__init__.py:243] Automatically detected platform cuda.


2025-06-03 06:06:48,741	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
model = '/root/Qwen/Qwen/Qwen3-8B'
tokenzier = AutoTokenizer.from_pretrained(model, use_fast=True)

prompt = "给我一个关于大模型的简短介绍。"
messages = [
    {
        "role": "user",
        "content": prompt
    }
]
text = tokenzier.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking = True   # 是否开启思考模式
)

outputs = get_completion(text, model, tokenizer=None, temperature=0.6, top_p = 0.95, top_k=20, min_p=0)  # 对于思考模式，官方建议使用以下参数：temperature = 0.6，TopP = 0.95，TopK = 20，MinP = 0。

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, \nResponse: {generated_text!r}\n")

INFO 06-03 06:13:09 [config.py:793] This model supports multiple tasks: {'score', 'classify', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 06-03 06:13:09 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-03 06:13:10 [core.py:438] Waiting for init message from front-end.
INFO 06-03 06:13:10 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='/root/Qwen/Qwen/Qwen3-8B', speculative_config=None, tokenizer='/root/Qwen/Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_ad

[W603 06:13:34.173447804 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W603 06:13:57.196061017 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[rank0]:[W603 06:13:57.267544441 ProcessGroupGloo.cpp:727] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


INFO 06-03 06:13:57 [parallel_state.py:1064] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
WARNING 06-03 06:13:57 [topk_topp_sampler.py:58] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
INFO 06-03 06:13:57 [gpu_model_runner.py:1531] Starting to load model /root/Qwen/Qwen/Qwen3-8B...
INFO 06-03 06:13:57 [cuda.py:217] Using Flash Attention backend on V1 engine.
INFO 06-03 06:13:58 [backends.py:35] Using InductorAdaptor
INFO 06-03 06:13:58 [backends.py:35] Using InductorAdaptor


Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:10<00:41, 10.39s/it]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:11<00:13,  4.66s/it]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:19<00:12,  6.38s/it]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:20<00:04,  4.14s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:20<00:00,  2.73s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:20<00:00,  4.08s/it]



INFO 06-03 06:14:18 [default_loader.py:280] Loading weights took 20.42 seconds
INFO 06-03 06:14:18 [gpu_model_runner.py:1549] Model loading took 15.2683 GiB and 20.705012 seconds
INFO 06-03 06:14:28 [backends.py:459] Using cache directory: /root/.cache/vllm/torch_compile_cache/1cb62ff5bb/rank_0_0 for vLLM's torch.compile
INFO 06-03 06:14:28 [backends.py:469] Dynamo bytecode transform time: 9.66 s
INFO 06-03 06:14:34 [backends.py:158] Cache the graph of shape None for later use
INFO 06-03 06:15:11 [backends.py:170] Compiling a graph for general shape takes 41.97 s
INFO 06-03 06:15:32 [monitor.py:33] torch.compile takes 51.63 s in total
INFO 06-03 06:15:33 [kv_cache_utils.py:637] GPU KV cache size: 31,072 tokens
INFO 06-03 06:15:33 [kv_cache_utils.py:640] Maximum concurrency for 8,192 tokens per request: 3.79x
INFO 06-03 06:16:06 [gpu_model_runner.py:1933] Graph capturing finished in 33 secs, took 0.59 GiB
INFO 06-03 06:16:06 [core.py:167] init engine (profile, create kv cache, warmup mo

Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.83s/it, est. speed input: 2.04 toks/s, output: 46.32 toks/s]

Prompt: '<|im_start|>user\n给我一个关于大模型的简短介绍。<|im_end|>\n<|im_start|>assistant\n', 
Response: '<think>\n好的，用户让我提供一个关于大模型的简短介绍。首先，我需要明确用户的需求是什么。他们可能对大模型不太了解，或者需要一个快速概述。我应该先确定大模型的基本定义，包括它们的规模、技术基础和应用场景。\n\n接下来，我需要考虑用户可能的背景。他们可能是学生、研究人员，或者对AI技术感兴趣的普通用户。如果是学生，可能需要更基础的解释；如果是研究人员，可能需要更深入的技术细节。但用户要求的是“简短介绍”，所以应该保持简洁，避免过于技术化的术语。\n\n然后，我需要涵盖大模型的关键点：参数量、训练数据、应用场景、优势和挑战。比如，大模型通常有数十亿到万亿参数，基于深度学习，使用大量文本数据训练，能够处理多种任务如语言理解、生成、翻译等。优势包括强大的泛化能力，但挑战如计算资源需求高、数据隐私问题等。\n\n还要注意结构是否清晰，分点是否有助于理解。可能需要使用项目符号或简短段落。同时，要确保语言通俗易懂，避免专业术语过多，必要时进行解释。\n\n另外，用户可能想知道大模型的实际应用，比如在自然语言处理、图像识别、医疗诊断等领域的例子。但因为要简短，可能需要提到几个主要应用领域即可。\n\n最后，检查是否有遗漏的重要信息，比如大模型的发展趋势，如多模态模型、小模型优化等，但可能超出简短介绍的范围。保持内容精炼，确保信息准确且全面。\n</think>\n\n大模型（Large Language Models）是基于深度学习的自然语言处理技术，通过海量文本数据训练，具备强大的语言理解和生成能力。它们通常包含数十亿到万亿个参数，能完成文本生成、翻译、问答、编程等复杂任务。近年来，大模型在医疗、金融、教育等领域广泛应用，但也面临计算成本高、数据隐私和伦理挑战等问题。'



## 创建兼容OpenAI API接口的服务器

`Qwen3-8B`是兼容`OpenAI API`协议，因此可以直接使用`vLLM`创建`OpenAI服务器`，默认会创建在 http://localhost:8000, 服务器当前一次托管一个模型，并实现列表模型、`completions`和`Chat completions`端口
- `completions`：是基于文本生成任务,模型会在给定一段体时候生成一段文本,适用于生成文章、故事和邮件
- `Chat completions`：面向对话任务, 模型需要理解和生成对话。用于对话机器人或对话系统
创建服务器可以指定模型磨成、模型路径、聊天模版等参数
- `--host`和`--port`参数指定地址
- `--model` 参数制定模型名称
- `--chat-template` 指定聊天模版
- `--served-model-name` 指定服务模型的名称。
- `--max-model-len` 指定模型的最大长度。
- `--enable-reasoning` 开启思考模式
- `--reasoning-parser` 指定如何解析模型生成的推理内容

可以使用以下命令到终端上,启动Qwen3-8B模型的API接口
```bash
VLLM_USE_MODELSCOPE=true vllm serve /root/Qwen/Qwen/Qwen3-8B --served-model-name Qwen3-8B --max_model_len 8192 --enable-reasoning --reasoning-parser deepseek_r1
```

可以使用`curl`命令查看当前模型列表
```bash
curl http://localhost:8000/v1/models
```

## 使用python脚本请求OpenAI Completions API

In [5]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="sk-xxx"
)

completion = client.chat.completions.create(
    model="Qwen3-8B",
    messages=[
         {"role": "user", "content": "我想问你，5的阶乘是多少？"}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='\n\n5的阶乘（记作5!）是将1到5的所有正整数相乘的结果：\n\n$$\n5! = 5 \\times 4 \\times 3 \\times 2 \\times 1 = 120\n$$\n\n**答案：120**', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content='\n嗯，用户问的是5的阶乘是多少。首先，我得确认阶乘的定义。阶乘就是从1乘到那个数，比如n的阶乘是n×(n-1)×...×1。所以5的阶乘应该是5×4×3×2×1对吧？\n\n不过，我得仔细算一遍，别算错了。先算5乘4是20，然后乘3是60，接着乘2是120，最后乘1还是120。对吧？嗯，没错，应该是120。\n\n不过，有没有可能用户问的是别的什么？比如，有时候可能会有其他数学概念被混淆，但阶乘一般就是这个定义。再想想，有没有可能用户输入有误？比如是不是问的是5的某个其他运算？不过问题明确说“阶乘”，所以应该没问题。\n\n再检查一遍计算步骤，确保没有遗漏。5×4=20，没错；20×3=60，对的；60×2=120，没错；120×1=120。是的，没错。所以答案应该是120。\n\n或者有没有可能用户对阶乘的定义不太清楚？比如，是否包括0？不过0的阶乘是1，但这里问的是5，所以没问题。总之，答案应该是120。\n')
